In [4]:
# importing depedencies
import os
import matplotlib.pyplot as plt
import pandas as pd
import scipy as st
import numpy as np
import hvplot.pandas
import geopandas as gpd
import requests
import datetime
import json
import folium
import pymongo
import plotly.graph_objects as go
from pprint import pprint

### import data

In [5]:
# Import Fire_data Polygon Shape data, fire_data.json (from the resources folder in storage) 
with open("resources/fire_data.json", "r") as f:
    firedata = json.load(f)
    fdf = gpd.GeoDataFrame.from_features(firedata)
    print(len(fdf))
    print(fdf.head())

6065
                                            geometry SRC_AGENCY      FIRE_ID  \
0  POLYGON Z ((-115.75479 49.79610 0.00000, -115....         BC  2016-N10037   
1  MULTIPOLYGON Z (((-114.97290 49.01062 0.00000,...         BC  2016-N10016   
2  POLYGON Z ((-114.94281 50.10509 0.00000, -114....         BC  2016-N10009   
3  POLYGON Z ((-115.72874 49.63289 0.00000, -115....         BC  2016-N10004   
4  POLYGON Z ((-115.75699 49.67369 0.00000, -115....         BC  2016-N10002   

  FIRENAME  YEAR  MONTH  DAY    REP_DATE DATE_TYPE OUT_DATE  ... WATER_REM  \
0     None  2016      5   13  2016-05-13      None     None  ...      None   
1     None  2016      4   20  2016-04-20      None     None  ...      None   
2     None  2016      4   13  2016-04-13      None     None  ...      None   
3     None  2016      4    5  2016-04-05      None     None  ...      None   
4     None  2016      4    2  2016-04-02      None     None  ...      None   

   UNBURN_REM  MORE_INFO   POLY_DATE      CFS

In [6]:
# Connecting to the MongoDB database (i.e. loading the Fire_data dataset through MongoDB into a variable)
def process_data():   
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client["Proj_3_Team_4_Data_Visualization"]
    collection = db["Fire_data"]
    cursor = collection.find()
    data = list(cursor)
    print(len(data))
    df = gpd.GeoDataFrame.from_features(data)
    print(df.head())
process_data()

6065
                                            geometry SRC_AGENCY      FIRE_ID  \
0  POLYGON Z ((-114.94281 50.10509 0.00000, -114....         BC  2016-N10009   
1  POLYGON Z ((-115.75479 49.79610 0.00000, -115....         BC  2016-N10037   
2  POLYGON Z ((-121.61167 50.23550 0.00000, -121....         BC  2016-K70443   
3  POLYGON Z ((-115.72874 49.63289 0.00000, -115....         BC  2016-N10004   
4  POLYGON Z ((-115.75699 49.67369 0.00000, -115....         BC  2016-N10002   

  FIRENAME  YEAR  MONTH  DAY    REP_DATE DATE_TYPE OUT_DATE  ... WATER_REM  \
0     None  2016      4   13  2016-04-13      None     None  ...      None   
1     None  2016      5   13  2016-05-13      None     None  ...      None   
2     None  2016      8   31  2016-08-31      None     None  ...      None   
3     None  2016      4    5  2016-04-05      None     None  ...      None   
4     None  2016      4    2  2016-04-02      None     None  ...      None   

   UNBURN_REM  MORE_INFO   POLY_DATE      CFS

In [7]:
# Import Weather Polygon Data, weather-provinces-combined.json (from the resources folder in storage) 
with open("resources/weather-provinces-combined.json", "r") as f:
    weather = json.load(f)
    weather_df = gpd.GeoDataFrame.from_features(weather,crs='EPSG:4617')
    print(len(weather_df))
    print(weather_df.head())
    print(weather_df.crs)

6500
                      geometry MIN_REL_HUMIDITY_FLAG  DIRECTION_MAX_GUST  \
0  POINT (-125.25417 71.99250)                  None                 0.0   
1  POINT (-125.25417 71.99250)                  None                 0.0   
2  POINT (-125.25417 71.99250)                  None                 0.0   
3  POINT (-125.25417 71.99250)                  None                 0.0   
4  POINT (-125.25417 71.99250)                  None                 0.0   

  COOLING_DEGREE_DAYS_FLAG HEATING_DEGREE_DAYS_FLAG DIRECTION_MAX_GUST_FLAG  \
0                     None                     None                    None   
1                        M                        M                    None   
2                        M                        M                    None   
3                        M                        M                    None   
4                        M                        M                    None   

  SPEED_MAX_GUST_FLAG  MAX_REL_HUMIDITY  MEAN_TEMPERATURE  \
0 

In [8]:
# Connecting to the MongoDB database (i.e. loading the Weather_Canada dataset through MongoDB into a variable)
def process_data():
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client["Proj_3_Team_4_Data_Visualization"]
    collection = db["Weather_Canada"]
    cursor = collection.find()
    data = list(cursor)
    print(len(data))
    weather_df = gpd.GeoDataFrame.from_features(data,crs='EPSG:4617')
    print(weather_df.head())
    print(weather_df.crs)
process_data()

6500
                      geometry MIN_REL_HUMIDITY_FLAG  DIRECTION_MAX_GUST  \
0  POINT (-125.25417 71.99250)                  None                 0.0   
1  POINT (-125.25417 71.99250)                  None                 0.0   
2  POINT (-125.25417 71.99250)                  None                 0.0   
3  POINT (-125.25417 71.99250)                  None                 0.0   
4  POINT (-125.25417 71.99250)                  None                 0.0   

  COOLING_DEGREE_DAYS_FLAG HEATING_DEGREE_DAYS_FLAG DIRECTION_MAX_GUST_FLAG  \
0                        M                        M                    None   
1                     None                     None                    None   
2                        M                        M                    None   
3                        M                        M                    None   
4                        M                        M                    None   

  SPEED_MAX_GUST_FLAG  MAX_REL_HUMIDITY  MEAN_TEMPERATURE  \
0 

In [9]:
# Import Fire Point Data (for large fires), fire_point_data.txt fire_data.json (from the resources folder in storage) 
fire_point_data_df =  pd.read_csv('resources/fire_point_data.txt')
fire_point_data_gdf = gpd.GeoDataFrame(fire_point_data_df, geometry=gpd.points_from_xy(fire_point_data_df.LONGITUDE, fire_point_data_df.LATITUDE), crs='EPSG:4617')
print(len(fire_point_data_gdf))
print(fire_point_data_gdf.head())

19115
   FID SRC_AGENCY      FIRE_ID FIRENAME  LATITUDE  LONGITUDE  YEAR  MONTH  \
0    0         BC  1950-G00026             59.876   -131.922  1950      6   
1    1         BC  1951-R00097             59.760   -132.808  1951      7   
2    2         BC  1950-G00035             57.463   -122.816  1950      6   
3    3         BC  1953-G00043             57.684   -121.925  1953      5   
4    4         BC  1950-R00029             59.318   -133.228  1950      6   

   DAY             REP_DATE  ...           CFS_REF_ID CFS_NOTE1 CFS_NOTE2  \
0    4  1950-06-04 00:00:00  ...  BC-1950-1950-G00026                       
1   15  1951-07-15 00:00:00  ...  BC-1951-1951-R00097                       
2   14  1950-06-14 00:00:00  ...  BC-1950-1950-G00035                       
3    1  1953-05-01 00:00:00  ...  BC-1953-1953-G00043                       
4   22  1950-06-22 00:00:00  ...  BC-1950-1950-R00029                       

              ACQ_DATE SRC_AGY2 ECOZONE ECOZ_REF          ECOZ_NAME 

In [10]:
# Connecting to the MongoDB database (i.e. loading the Fire_Point dataset through MongoDB into a variable)
def process_data():
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client["Proj_3_Team_4_Data_Visualization"]
    collection = db["Fire_data_collection"]
    cursor = collection.find()
    data = list(cursor)
    print(len(data))
    fire_point_df = gpd.GeoDataFrame(fire_point_data_df, geometry=gpd.points_from_xy(fire_point_data_df.LONGITUDE, fire_point_data_df.LATITUDE), crs='EPSG:4617')
    print(fire_point_df.head())
process_data()

19115
   FID SRC_AGENCY      FIRE_ID FIRENAME  LATITUDE  LONGITUDE  YEAR  MONTH  \
0    0         BC  1950-G00026             59.876   -131.922  1950      6   
1    1         BC  1951-R00097             59.760   -132.808  1951      7   
2    2         BC  1950-G00035             57.463   -122.816  1950      6   
3    3         BC  1953-G00043             57.684   -121.925  1953      5   
4    4         BC  1950-R00029             59.318   -133.228  1950      6   

   DAY             REP_DATE  ...           CFS_REF_ID CFS_NOTE1 CFS_NOTE2  \
0    4  1950-06-04 00:00:00  ...  BC-1950-1950-G00026                       
1   15  1951-07-15 00:00:00  ...  BC-1951-1951-R00097                       
2   14  1950-06-14 00:00:00  ...  BC-1950-1950-G00035                       
3    1  1953-05-01 00:00:00  ...  BC-1953-1953-G00043                       
4   22  1950-06-22 00:00:00  ...  BC-1950-1950-R00029                       

              ACQ_DATE SRC_AGY2 ECOZONE ECOZ_REF          ECOZ_NAME 

In [13]:

WEATHER_API_BASE = "https://api.weather.gc.ca"
fire_points_df = pd.read_csv(os.path.join('resources','fire_point_data.txt'))
def get_weather_for_day(date: datetime.datetime) -> gpd.GeoDataFrame:
  """
  Load weather data from all stations for a single day.
  """
  API = f"{WEATHER_API_BASE}/collections/climate-daily/items"

  date_start = date.__format__("%Y-%m-%d 00:00:00")
  date_end = date.__format__("%Y-%m-%d 11:59:59")

  result = requests.get(API, params={
    "datetime": f"{date_start}/{date_end}",
    "f": "json",
    "startindex": "0",
    "sortby": "LOCAL_YEAR,LOCAL_DAY",
    "limit": "5000"
  })

  data = result.json()
  features = data["features"]
  df = gpd.GeoDataFrame.from_features(features)

  return df

def get_fires_for_day(date: datetime.datetime) -> pd.DataFrame:
  """
  Get a subset of the fire points data for a single day.
  """
  df = fire_points_df[(fire_points_df["YEAR"] == date.year) & (fire_points_df["MONTH"] == date.month) & (fire_points_df["DAY"] == date.day)]
  return df

def make_weather_and_fire_animation(start_date, end_date):
  """
  Make an animation of weather and fire data over a range of dates.
  """
  # Load fire data from CSV.
  with open(os.path.join('resources','fire_point_data.txt')) as f:
    fire_points = pd.read_csv(f)

    # Iterate through days in range.
    for i, date in enumerate(pd.date_range(start_date, end_date)):
      print(f"Processing date: {date}")

      weather_for_day_df = get_weather_for_day(date)
      fires_for_date_df = get_fires_for_day(date)

      # Create a plot with all weather data color coded from blue-red (relative temperatures)
      # Overlay with the forest fires for that day, in black.
      fig, ax = plt.subplots()
      plt.title(f"{date}")
      weather_for_day_df.plot("MEAN_TEMPERATURE", cmap="coolwarm", ax=ax)
      ax.scatter(fires_for_date_df["LONGITUDE"], fires_for_date_df["LATITUDE"], color="black", s=10)
      
      # Save frame of animation to animations folder.
      plt.savefig(f"resources/animation/{i}.png")
      plt.close()


#print(len(weather_df))
#print(len(fire_point_data_gdf))
#plt.show()
#ax.legend(loc='upper right')

In [12]:
START_DATE = datetime.datetime(2020, 5, 1)
END_DATE = datetime.datetime(2020, 5, 31)
make_weather_and_fire_animation(START_DATE, END_DATE)


Processing date: 2020-05-01 00:00:00
Processing date: 2020-05-02 00:00:00
Processing date: 2020-05-03 00:00:00
Processing date: 2020-05-04 00:00:00
Processing date: 2020-05-05 00:00:00
Processing date: 2020-05-06 00:00:00
Processing date: 2020-05-07 00:00:00
Processing date: 2020-05-08 00:00:00
Processing date: 2020-05-09 00:00:00
Processing date: 2020-05-10 00:00:00
Processing date: 2020-05-11 00:00:00
Processing date: 2020-05-12 00:00:00
Processing date: 2020-05-13 00:00:00
Processing date: 2020-05-14 00:00:00
Processing date: 2020-05-15 00:00:00
Processing date: 2020-05-16 00:00:00
Processing date: 2020-05-17 00:00:00
Processing date: 2020-05-18 00:00:00
Processing date: 2020-05-19 00:00:00
Processing date: 2020-05-20 00:00:00
Processing date: 2020-05-21 00:00:00
Processing date: 2020-05-22 00:00:00
Processing date: 2020-05-23 00:00:00
Processing date: 2020-05-24 00:00:00
Processing date: 2020-05-25 00:00:00
Processing date: 2020-05-26 00:00:00
Processing date: 2020-05-27 00:00:00
P